<a href="https://colab.research.google.com/github/guilhermefritz/SQL_Projects/blob/main/SQL_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Feito por Guilherme Fritz

## Usando o DuckDB para executar consultas  SQL dentro de um ambiente Python.

DuckDB-É um sistema de gerenciamento de base de dados SQL que roda no mesmo processo em que o aplicativo que utiliza o banco de dados,o que significa que não se faz necessária a criação de uma rede de comunicação entre os dois, acelerando o acesso aos dados armazenados

Dataset de vendas  disponivel  em https://www.kaggle.com/datasets/rohitsahoo/sales-forecasting

### A partir de uma análise minuciosa dos dados históricos de vendas no período de 2016 a 2018, obtive uma valiosa fonte de conhecimento que fundamenta o projeto  

### A riqueza dessas informações não apenas ilumina os eventos passados, mas também serve como uma bússola para as decisões futuras. . 

-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

# **Perguntas a serem respondidas**:


  *  #### 1-Qual foi a quantidade de pedidos no total?
  
  *  #### 2-Qual foi o valor máximo de um pedido  e o valor mínimo de um pedido entre todos os pedidos?
  
  * ####  3-De 2016 a 2018,quais foram os produtos TOP1(Lideraram o ranking) em valor de vendas de cada ano?
  
  * ####  4-Qual foi o semestre dos anos de 2017 e 2018 que obteve o maior faturamento ? Nesse determinado semestre quais foram os TOP2 produtos em vendas?
  
  * ####  5-Qual foram os estados que tiveram faturamento >70.000? Traga a media de valor gasto por cada cidade 
  
  * ####  6-Qual foi o Estado  campeão de faturamento ao decorrer de todos os anos ? Quantos clientes distintos esse estado teve?
  
  * ####  7-Desse  determinado estado,quais foram as TOP3 cidades campeãs de vendas ?
  
  * ####  8-Qual foi a media de valor de venda agrupando por estado e cidade?
  
  * ####  9-As categorias de produtos sao divididas entre quais tipos ?
  
  * ####  10-Quanto cada categoria vendeu nos  TOP3 estados? 
  
  * ####  11-Supondo que (Tecnologia) é a categoria dominante em valor de  vendas, teve algum ano em que outra categoria a sobressaiu?
  
  * ####  12-Da categoria de (Tecnologia) ,qual das sub-categorias se sobressaiu das demais?Dessa categoria quantos clientes eram distintos?
  
  * ####  13-Qual foi o tipo vencedor do ship mode(modo de envio) mais preferido pelos clientes
  
  * ####  14 -Qual foi o produto  que mais faturou no decorrer dos anos?
  
  * ####  15-Veja a lista de compradores desse produto e o respectivo codigo postal de cada um .
  
  * ####  16-Teve algum ano em que no mês de Novembro a quantidade de faturamento  foi superior que o mês de Dezembro?
  
  * ####  17-Quais foram os TOP3 clientes que mais gastaram  na loja  ?






In [1]:
 pip install duckdb


Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd

In [18]:
df=pd.read_csv("D:datasets/Sales_forecast/train.csv")
#Lendo o arquivo que contém o dataset

In [19]:
df.info()
#Informaçoes do dataset

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9800 entries, 0 to 9799
Data columns (total 19 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Unnamed: 0     9800 non-null   int64  
 1   Row ID         9800 non-null   int64  
 2   Order ID       9800 non-null   object 
 3   Order Date     9800 non-null   object 
 4   Ship Date      9800 non-null   object 
 5   Ship Mode      9800 non-null   object 
 6   Customer ID    9800 non-null   object 
 7   Customer Name  9800 non-null   object 
 8   Segment        9800 non-null   object 
 9   Country        9800 non-null   object 
 10  City           9800 non-null   object 
 11  State          9800 non-null   object 
 12  Postal Code    9789 non-null   float64
 13  Region         9800 non-null   object 
 14  Product ID     9800 non-null   object 
 15  Category       9800 non-null   object 
 16  Sub-Category   9800 non-null   object 
 17  Product Name   9800 non-null   object 
 18  Sales   

In [20]:
df.head()
#Visualizando o dataset

,Unnamed: 0,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,State,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales
0,0,1,CA-2017-152156,2017-11-08,11/11/2017,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420.0,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600
1,1,2,CA-2017-152156,2017-11-08,11/11/2017,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420.0,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400
2,2,3,CA-2017-138688,2017-06-12,16/06/2017,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,California,90036.0,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200
3,3,4,US-2016-108966,2016-10-11,18/10/2016,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,33311.0,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775
4,4,5,US-2016-108966,2016-10-11,18/10/2016,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,33311.0,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680


In [21]:
import duckdb

### Para extrair a data no Tipo 'Date' o Duck db requer que esteja nesse formato (YYYY-MM-DD) como o formato original do dataset estava (DD/MM/YYYY) foi necessário alterar.

In [ ]:
df['Order Date'] = pd.to_datetime(df['Order Date'], format='%d/%m/%Y')
# Salve os dados atualizados em um novo arquivo CSV
df.to_csv("D:datasets/Sales_forecast/train_att.csv")

In [ ]:
df=pd.read_csv("D:datasets/Sales_forecast/train_att.csv")

In [22]:
#Estabelecendo conexão com o duck db
# 'clientes' agora refere-se a 'df'(dataset)
conexao = duckdb.connect(database=":memory:", read_only=False)
conexao.register("vendas", df)


## **1º pergunta:**

In [28]:
#contagem 
query_1='''
SELECT COUNT(*) as contagem_pedidos
FROM vendas
;





'''
resultado_x = conexao.execute(query_x).df()
print(resultado_x)

   contagem_pedidos
0              9800


## **2º pergunta:**

In [120]:
#valor maximo 
query_2 ='''
SELECT "Product Name",Sales AS maior_valor
FROM vendas
WHERE Sales = (SELECT MAX(Sales) FROM vendas);

    '''

    
resultado_2 = conexao.execute(query_2).df()
print(resultado_2)

                                        Product Name  maior_valor
0  Cisco TelePresence System EX90 Videoconferenci...     22638.48


In [121]:
#valor min
query_2_1 ='''

SELECT "Product Name",Sales AS menor_valor
FROM  vendas
WHERE "Sales" = (SELECT MIN(Sales) FROM vendas);

    '''
resultado_2_1 = conexao.execute(query_2_1).df()
print(resultado_2_1)

                                        Product Name  menor_valor
0  Hoover Replacement Belt for Commercial Guardsm...        0.444


## **3º pergunta:**

In [ ]:
#produto 2016
query3_1 ='''
SELECT
 EXTRACT(YEAR FROM CAST("Order Date" AS DATE)) AS Ano,
"Product Name" AS ProdutoMaisVendido,
SUM(Sales) AS ValorTotal
FROM vendas
WHERE EXTRACT(YEAR FROM CAST("Order Date" AS DATE)) = 2017
GROUP BY Ano, "Product Name"
ORDER BY Ano DESC, ValorTotal DESC LIMIT 1






'''
resultado_3 = conexao.execute(query_3).df()
print(resultado_3)

2016 : 🥇'Fellowes PB500 Electric...'

In [80]:
#produto 2017
query3_2 ='''
SELECT
 EXTRACT(YEAR FROM CAST("Order Date" AS DATE)) AS Ano,
"Product Name" AS ProdutoMaisVendido,
SUM(Sales) AS ValorTotal
FROM vendas
WHERE EXTRACT(YEAR FROM CAST("Order Date" AS DATE)) = 2017
GROUP BY Ano, "Product Name"
ORDER BY Ano DESC, ValorTotal DESC LIMIT 1






'''
resultado_3 = conexao.execute(query_3).df()
print(resultado_3)

    Ano                     ProdutoMaisVendido  ValorTotal
0  2017  Canon imageCLASS 2200 Advanced Copier   25899.926


2017:*🥇* 'Canon imageCLASS 2200 Advanced **Copier** '

In [79]:
#produto 2018 
query3_3 ='''
SELECT
 EXTRACT(YEAR FROM CAST("Order Date" AS DATE)) AS Ano,
"Product Name" AS ProdutoMaisVendido,
SUM(Sales) AS ValorTotal
FROM vendas
WHERE EXTRACT(YEAR FROM CAST("Order Date" AS DATE)) = 2018
GROUP BY Ano, "Product Name"
ORDER BY Ano DESC, ValorTotal DESC LIMIT 1






'''
resultado_3 = conexao.execute(query_3).df()
print(resultado_3)

    Ano                     ProdutoMaisVendido  ValorTotal
0  2018  Canon imageCLASS 2200 Advanced Copier   35699.898


2018:🥇'Canon imageCLASS 2200 Advanced Copier'


## **4º pergunta:**

In [81]:
#semestre com maior faturamento 
query_4 ='''
SELECT
  EXTRACT(YEAR FROM CAST("Order Date" AS DATE)) AS Ano,
CASE
  WHEN EXTRACT(MONTH FROM CAST("Order Date" AS DATE)) BETWEEN 1 AND 6 THEN '1º Semestre'
  WHEN EXTRACT(MONTH FROM CAST("Order Date" AS DATE)) BETWEEN 7 AND 12 THEN '2º Semestre'
  END AS Semestre,
SUM(Sales) AS TotalVendas
FROM vendas
WHERE EXTRACT(YEAR FROM CAST("Order Date" AS DATE)) IN (2017, 2018)
GROUP BY Ano, Semestre
ORDER BY Ano, Semestre;




'''
resultado_4 = conexao.execute(query_4).df()
print(resultado_4)

    Ano     Semestre  TotalVendas
0  2017  1º Semestre  227747.5260
1  2017  2º Semestre  372445.0240
2  2018  1º Semestre  249819.5042
3  2018  2º Semestre  472232.5150


## **5º pergunta:**

In [123]:
#total de vendas agrupando por estado quando o valor total for maior que 70.000
query_5 = '''
SELECT State,SUM(Sales) AS total_vendas
FROM vendas
GROUP BY  State
HAVING total_vendas > 70000
ORDER BY total_vendas DESC
LIMIT 10
'''

resultado_5 = conexao.execute(query_5).df()
print(resultado_5)

          State  total_vendas
0    California   446306.4635
1      New York   306361.1470
2         Texas   168572.5322
3    Washington   135206.8500
4  Pennsylvania   116276.6500
5       Florida    88436.5320
6      Illinois    79236.5170
7      Michigan    76136.0740
8          Ohio    75130.3500
9      Virginia    70636.7200


## **6º pergunta:**

In [110]:
# Qual foi o estado campeao em total de vendas
query6 ='''
SELECT State,SUM(Sales) AS totalsales
FROM vendas
GROUP BY State
ORDER BY totalsales DESC LIMIT 10;


'''
resultado6 = conexao.execute(query6).df()
print(resultado6)

          State   totalsales
0    California  446306.4635
1      New York  306361.1470
2         Texas  168572.5322
3    Washington  135206.8500
4  Pennsylvania  116276.6500
5       Florida   88436.5320
6      Illinois   79236.5170
7      Michigan   76136.0740
8          Ohio   75130.3500
9      Virginia   70636.7200


 Estado da California *🥇*

In [111]:
#Qual foi a quantidade de clientes distintos na California?
query_6_1 = '''
SELECT State, COUNT(DISTINCT "Customer ID") AS total_clientes
FROM vendas
WHERE State = 'California'
GROUP BY State;
'''

resultado_6_1 = conexao.execute(query_6_1).df()
print(resultado_6_1)



        State  total_clientes
0  California             570


## **7º pergunta:**

In [162]:
#top 3 cidades campeãs do estado
query_7 ='''
SELECT City,SUM(Sales) AS totalsales
FROM vendas
WHERE State = 'California'
GROUP BY City
ORDER BY totalsales DESC
LIMIT 3;

'''
resultado_7 = conexao.execute(query_7).df()
print(resultado_7)

            City  totalsales
0    Los Angeles  173420.181
1  San Francisco  109041.120
2      San Diego   47521.029


## **8º pergunta:**

In [112]:
#Media de pedidos agrupando por estado e cidade
query_8 = '''
SELECT State,City,AVG(Sales) AS media_pedidos
FROM vendas
GROUP BY State,City
ORDER BY media_pedidos DESC
LIMIT 10;
'''

resultado_8 = conexao.execute(query_8).df()
print(resultado_e)

            State          City  media_pedidos
0        New York     Jamestown    2354.395000
1         Indiana     Lafayette    1784.046364
2         Wyoming      Cheyenne    1603.136000
3      Washington    Bellingham    1263.413333
4        Missouri  Independence    1208.685000
5  North Carolina    Burlington    1152.843818
6      California       Burbank    1082.386000
7        New York       Buffalo     906.349600
8   Massachusetts       Beverly     861.063333
9          Nevada        Sparks     853.986667


## **9º pergunta:**

In [113]:
#categoria produtos
query_9 ='''
SELECT DISTINCT Category,
FROM vendas
;

'''
resultado_9 = conexao.execute(query_9).df()
print(resultado_9)

          Category
0  Office Supplies
1        Furniture
2       Technology


## **10º pergunta:**

In [124]:
#Qual categoria vendeu mais entre os top3 estados em valor de vends 
query_10 ='''
SELECT
Category,
 SUM(CASE WHEN State = 'California' THEN Sales ELSE 0 END) AS TotalSales_California,
 SUM(CASE WHEN State = 'New York' THEN Sales ELSE 0 END) AS TotalSales_NY,
 SUM(CASE WHEN State = 'Texas' THEN Sales ELSE 0 END) AS TotalSales_Texas
FROM vendas
WHERE State IN ('California', 'New York','Texas')
GROUP BY Category
ORDER BY Category DESC ;

'''
resultado_10 = conexao.execute(query_10).df()
print(resultado_10)

          Category  TotalSales_California  TotalSales_NY  TotalSales_Texas
0       Technology            154684.1800     126902.690        64656.2720
1  Office Supplies            139405.7480      86953.892        44282.6160
2        Furniture            152216.5355      92504.565        59633.6442


Categoria de Tecnologia 🥇

## **11º pergunta:**

In [88]:
#mudança de TOP categoria
query_11='''


SELECT "Category" AS CategoriaMaisVendida, SUM(Sales) AS TotalVendas
FROM vendas
WHERE EXTRACT(YEAR FROM CAST("Order Date" AS DATE)) = 2016
GROUP BY "Category"
ORDER BY TotalVendas DESC
LIMIT 3;




'''
resultado_11 = conexao.execute(query_11).df()
print(resultado_11)

  CategoriaMaisVendida  TotalVendas
0            Furniture  164053.8674
1           Technology  162257.7310
2      Office Supplies  133124.4070


Após analisar cada ano ,descubro que o unico ano em que a categoria de 'Tecnologia' não foi a campeã em valor de vendas foi em 2016 em que a categoria 'Furniture" foi a vencedora!

## **12 pergunta:**

In [90]:
#sub_categoria
query_12='''
SELECT "Sub-Category" , SUM(Sales) AS TotalVendas
FROM vendas
WHERE Category = 'Technology'
GROUP BY "Sub-Category"
ORDER BY TotalVendas DESC
LIMIT 1;
'''
resultado_12 = conexao.execute(query_12).df()
print(resultado_12)

  Sub-Category  TotalVendas
0       Phones   327782.448


Sub Categoria 'Phones' 🥇

In [75]:
#clientes distintos que compraram 'phones'
query_12_1 = '''
SELECT "Sub-Category",COUNT(DISTINCT "Customer ID") AS total_clientes
FROM vendas
WHERE Category = 'Technology' AND "Sub-Category" = 'Phones'
GROUP BY "Sub-Category";
'''

resultado_12_1 = conexao.execute(query_12_1).df()
print(resultado_12_1)

  Sub-Category  total_clientes
0       Phones             509


## **13º pergunta:**

In [91]:
#tipo de metodo de  entrega vencedor
query_13 ='''
SELECT "Ship Mode", COUNT(*) AS Quantidade
FROM vendas
WHERE "Ship Mode" IN ('Standard Class', 'Second Class')
GROUP BY "Ship Mode"
ORDER BY Quantidade DESC;

'''
resultado_13 = conexao.execute(query_13).df()
print(resultado_13)

        Ship Mode  Quantidade
0  Standard Class        5859
1    Second Class        1902


## **14º pergunta:**

In [92]:
#produto que mais faturou em todos os anos
query_14='''
SELECT "Product Name" AS ProdutoCampeão,SUM(Sales) AS TotalVendas
FROM vendas
GROUP BY "Product Name"
ORDER BY TotalVendas DESC
LIMIT 1;

'''

resultado_14 = conexao.execute(query_14).df()
print(resultado_14)

                          ProdutoCampeão  TotalVendas
0  Canon imageCLASS 2200 Advanced Copier    61599.824


Produto que mais vendeu em valor foi 🥇: 'Canon imageCLASS 2200 Advanced Copier'

>



## **15º pergunta:**

Podemos supor que o motivo de saber o nome e o codigo postal da pessoa pode ser para enviar um voucher de desconto para essas pessoas adquirirem outro produto da loja.

In [93]:

#cod postal
query_15='''

SELECT "Customer Name","Postal Code", SUM(Sales)
FROM vendas
WHERE "Product Name" = 'Canon imageCLASS 2200 Advanced Copier'
GROUP BY "Customer Name","Postal Code"
;
'''
resultado_15= conexao.execute(query_15).df()
print(resultado_15)

        Customer Name  Postal Code  sum(Sales)
0        Hunter Lopez      19711.0   10499.970
1  Christopher Conant      19120.0    8399.976
2        Tamara Chand      47905.0   17499.950
3        Raymond Buch      98115.0   13999.960
4        Tom Ashbrook      10024.0   11199.968


## **16º pergunta:**

O motivo de se querer comparar se houve algum  mês de novembro de um dos anos  em que o faturamento foi maior que o mês de dezembro pode ser para comparação  de vendas,já que em dezembro as lojas costumam ter (evento de natal)  onde se tem promoções em determinados  produtos da loja.

Após uma analise percebo que apenas em 2016 teve mais vendas no mês 11 que no mês 12, onde podemos supor  que  a partir de 2017 a loja veio com propostas de ofertas imperdiveis para o natal o que fez  subir o faturamento comparado com o mês anterior.

In [94]:
#mes
query_16 ='''
SELECT
 EXTRACT(YEAR FROM CAST("Order Date" AS DATE)) AS Ano,
 EXTRACT(MONTH FROM CAST("Order Date" AS DATE)) AS Mes,
 SUM(Sales) AS TotalVendas
FROM vendas
WHERE EXTRACT(MONTH FROM CAST("Order Date" AS DATE)) =12
AND EXTRACT(YEAR FROM CAST("Order Date" AS DATE)) = 2016
GROUP BY Ano, Mes
ORDER BY Ano DESC, Mes DESC;



'''
resultado_16 = conexao.execute(query_16).df()
print(resultado_16)

    Ano  Mes  TotalVendas
0  2016   12   74543.6012


## **17º pergunta:**

 Podemos supor que os TOP 3 clientes que mais consumiram na loja vão receber em sua casa uma bicicleta.

In [114]:
#presente
query_17='''
SELECT "Customer ID", "Customer Name","Postal Code", SUM(Sales) AS TotalGasto
FROM vendas
GROUP BY "Customer ID", "Customer Name", "Postal Code"
ORDER BY TotalGasto DESC
LIMIT 3;
'''
resultado_17 = conexao.execute(query_17).df()
print(resultado_17)

  Customer ID Customer Name  Postal Code  TotalGasto
0    SM-20320   Sean Miller      32216.0   23661.228
1    TC-20980  Tamara Chand      47905.0   18336.740
2    RB-19360  Raymond Buch      98115.0   14052.480


# FIM